In [12]:
# Hello world!
# Below is the effect of my first ever Python project
# based on the book by Tariq Rashid "How to make your own neural network"
# Completed between 10 and 27 Dec 2019
# it may not seem like much considering that I basically followed a ready recipe
# but I tried my best to understand the concepts
# "it ain't much, but it's honest work"

In [ ]:
# to do operations on matrices etc
import numpy
#for the sigmoid activation function
import scipy.special
# library for plotting arrays
import matplotlib.pyplot
#to plot arrays in this notebook

#neural network class definition
%matplotlib inline
class neuralNetwork:
    #initializing function
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        #setting number of nodes in each layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        #link weights matrices between the nodes in different layers
        #wih between input and hidden layer
        #who between hidden and output
        #this function turns the links into a matrix
        self.wih = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))
        self.lr = learningrate
        #activation function is the sigmoid function 
        self.activation_function = lambda x: scipy.special.expit(x)
        pass
    def train(self, inputs_list, targets_list):
        #convert inputs and targets into a 2d array
        inputs = numpy.array(inputs_list,ndmin=2).T
        targets = numpy.array(targets_list,ndmin=2).T
        #calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        #calculate signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        #calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        #calculate signals emerging from the final output layer
        final_outputs = self.activation_function(final_inputs)
        
        #ok until now it was the same as def query, ie calcuclating the output
        #now it's time to calculate the error, which is (t-o)^2
        #targets - final_outputs
        output_errors = targets - final_outputs
        hidden_errors = numpy.dot(self.who.T, output_errors)
        #update the weight links for the hidden and final layers
        self.who += self.lr * numpy.dot((output_errors*final_outputs*(1.0-final_outputs)), numpy.transpose(hidden_outputs))
        #now update the link weights for wih (input to hidden)
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        pass
    def query(self, inputs_list):
        #convert inputs list to a 2d array
        inputs = numpy.array(inputs_list,ndmin=2).T
        # calculate signals into the hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        #apply activation function to the hidden outputs
        hidden_outputs = self.activation_function(hidden_inputs)
        #calculate signals into the output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        #apply activation function to the final outputs
        final_outputs = self.activation_function(final_inputs)
        return final_outputs
                                        
#stating nr of nodes in each layer
input_nodes = 784
hidden_nodes = 100
output_nodes = 10

#stating the learning rate
learning_rate = 0.3

#creating an object n belonging to the neuralNetwork class
n = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

#loading training data mnist in CSV file
training_data_file = open("mnist_dataset/mnist_train.csv", "r")
training_data_list = training_data_file.readlines()
training_data_file.close()

#training the neural network
#split the text in the csv training files by commas
for record in training_data_list:
    #split the record by commas
    all_values = record.split(',')
    #scale and shift the inputs
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    #create the target outputs values (all 0.01 except for the desired label which is 0.99)
    targets = numpy.zeros(output_nodes) + 0.01
    #all_values[0] is the label record for this record
    targets[int(all_values[0])] = 0.99
    n.train(inputs,targets)
    pass
pass

In [13]:
#testing the network
test_data_file = open("mnist_dataset/mnist_test.csv", "r")
test_data_list = test_data_file.readlines()
test_data_file.close()

In [18]:
#test the neural network
#scorecard for how well the network performs, initially empty scorecard []
# go through all the records in the test data set
scorecard = []
for record in test_data_list:
    #split record by commas
    all_values = record.split(',')
    #correct answer is first value
    correct_label = int(all_values[0])
    #scale and shift the inputs
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    #query the network
    outputs = n.query(inputs)
    #the index of the highest value corresponds to the label
    label = numpy.argmax(outputs)
    #append correct or incorrect to the list
    if (label == correct_label):
    #if network's answer matches the correct answer,add 1 to the scorecard
        scorecard.append(1)
    #if network's answer doesn't match the correct answer, add 0 to the scorecard
    else:
        scorecard.append(0)
        pass
    pass

#calculate the performance score, a fraction of correct answers
scorecard_array = numpy.asarray(scorecard)
print("performance =", scorecard_array.sum() / scorecard_array.size)

performance = 0.9427
